# Fonction renvoyant une durée restée dans un lieu selon l'heure et le lieu

In [74]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import ast
from scipy.stats import invgamma,loggamma,invgauss

!pip install openpyxl

In [75]:
#Lire le tableau des lois de durée restée dans un lieu:
tableau_duree= pd.read_excel("data/lois_duree.xlsx")

In [76]:
tableau_duree=tableau_duree.set_index("Unnamed: 0").rename_axis("Plage horaire")

In [77]:
tableau_duree=tableau_duree.map(lambda x: ast.literal_eval(x))


In [78]:
tableau_duree.head()

,Rue,Entreprise,Sans,Domicile,Parking
Plage horaire,,,,,
Matin,"[invgauss, (4.036204719120207, -0.161617635419...","[beta, (52.595462995358545, 27.703319219454222...","[beta, (0.5117940197816365, 19.2230902247954, ...","[beta, (0.5474258078179828, 0.9000121427405545...","[invgamma, (1.2485500680449202, -0.23360330017..."
Midi,"[chi2, (1.3400808769713695, -5.127648667537803...","[invgamma, (54.707261432752674, -13.6736528297...","[beta, (0.30450496225678947, 88.52544141212059...","[beta, (0.547862902931137, 0.6163748511340661,...","[pareto, (3.474502202630147, -5.75279416070615..."
Après-midi,"[chi2, (1.3380657374339826, -2.022860775150514...","[gamma, (1.1060959455943804, -0.00407001505164...","[beta, (0.8611652542892512, 79.35796489004701,...","[beta, (1.8969335796940112, 1.1440186443623233...","[invgamma, (2.030103317853264, -0.273339157562..."
Soir,"[beta, (0.7222192851774487, 1.4655167671346607...","[beta, (0.8699981002941614, 1.3044457209426563...","[beta, (0.3021326639337568, 266.1616492102139,...","[beta, (1.9493353130904008, 1.1828119830488304...","[invgauss, (1.3223013343950307, -0.16386888664..."


In [106]:
def duree_lieu(heure_arrivee,lieu):
    if heure_arrivee<=11:
        plage= "Matin"
    elif heure_arrivee<=14:
        plage="Midi"
    elif heure_arrivee<=17:
        plage= "Après-midi"
    else:
        plage= "Soir"
    loi=tableau_duree.loc[plage,lieu]
    dist=loi[0]
    #print(dist)
    param=list(loi[1])
    if abs(param[-2])<10**(-2):
        param[-2]=0
    #print(param)
    sample = getattr(stats,dist).rvs(*param)
    return sample

In [107]:
duree_lieu(9,"Sans")

0.14273439741416566